1. IMPORTS Y CONFIG. MODELO

In [1]:
# =========================================================================
# RAG MEJORADO - Corrección de datos y optimización
# Basado en: "A fine-tuning enhanced RAG system" (Rangan & Yin)
# =========================================================================

import pandas as pd
import chromadb
import re
from pathlib import Path
import time

from llama_index.core import Document, VectorStoreIndex, StorageContext, Settings
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.vector_stores import MetadataFilters, ExactMatchFilter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.prompts import PromptTemplate

# Configurar modelos
Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

Settings.llm = Ollama(
    model="llama3",
    request_timeout=120.0,
    system_prompt=(
        "Eres un asistente experto en documentos legales peruanos. "
        "SIEMPRE respondes en español. "
        "Usas únicamente la información proporcionada. "
        "Nunca inventes información."
    )
)

print("Imports configurados")
print("Metodología basada en: Rangan & Yin (2024) - RAG + Fine-tuning")

W0110 15:17:38.528000 9072 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.



Imports configurados
Metodología basada en: Rangan & Yin (2024) - RAG + Fine-tuning


2. CARGA Y MEJORA DE CSV

" Al Intentar extracción estructurada con regex (17.9% éxito) detectamos limitaciones. Identificamos que el 82% de los registros tenían información fragmentada que no permitía extracción estructurada con regex. Implementamos un enfoque donde el LLM extrae directamente del texto completo, que es más robusto y la técnica es respaldada por papers recientes (Rangan & Yin, 2024)."

In [2]:
# =========================================================================
# CARGAR Y MEJORAR CSV
# =========================================================================

print("="*70)
print("CARGANDO CSV ORIGINAL")
print("="*70 + "\n")

# Cargar CSV actual
df = pd.read_csv("base_datos_final/base_datos_completa.csv", low_memory=False)
print(f"Total registros: {len(df):,}")

# Filtrar tipos relevantes
tipos_relevantes = ['JUNTA_ACCIONISTAS', 'DISOLUCION', 'REMATE']
df_filtrado = df[df['tipo'].isin(tipos_relevantes)].copy()
print(f"Registros relevantes: {len(df_filtrado):,}\n")

print("Distribución por tipo:")
print(df_filtrado['tipo'].value_counts())

# APLICAR EXTRACCIÓN DE EMPRESAS
print("\n" + "="*70)
print("EXTRAYENDO EMPRESAS DEL TEXTO COMPLETO")
print("="*70 + "\n")

def extraer_empresa_del_texto(texto, tipo):
    """
    Extrae nombre real usando regex
    """
    if not isinstance(texto, str) or len(texto) < 20:
        return None
    
    texto = texto.replace('\n', ' ')
    
    if tipo == 'DISOLUCION':
        patrones = [
            r'disolución\s+y\s+liquidación\s+de\s+([A-Z][A-ZÁÉÍÓÚÑa-záéíóúñ\s\.\-&]+?(?:S\.A\.C\.|S\.A\.|E\.I\.R\.L\.|S\.R\.L\.))',
            r'empresa\s+denominada\s+([A-Z][A-ZÁÉÍÓÚÑa-záéíóúñ\s\.\-&]+?(?:S\.A\.C\.|S\.A\.|E\.I\.R\.L\.|S\.R\.L\.))',
            r'sociedad\s+([A-Z][A-ZÁÉÍÓÚÑa-záéíóúñ\s\.\-&]+?(?:S\.A\.C\.|S\.A\.|E\.I\.R\.L\.|S\.R\.L\.))',
            r'de\s+([A-Z][A-ZÁÉÍÓÚÑa-záéíóúñ\s\.\-&]+?(?:S\.A\.C\.|S\.A\.|E\.I\.R\.L\.|S\.R\.L\.))\s+con\s+RUC',
        ]
    elif tipo == 'JUNTA_ACCIONISTAS':
        patrones = [
            r'junta\s+.*?de\s+([A-Z][A-ZÁÉÍÓÚÑa-záéíóúñ\s\.\-&]+?(?:S\.A\.C\.|S\.A\.|E\.I\.R\.L\.|S\.R\.L\.))',
            r'empresa\s+([A-Z][A-ZÁÉÍÓÚÑa-záéíóúñ\s\.\-&]+?(?:S\.A\.C\.|S\.A\.|E\.I\.R\.L\.|S\.R\.L\.))',
        ]
    elif tipo == 'REMATE':
        patrones = [
            r'ubicado\s+en\s+([^,\.]{10,100})',
            r'ubicación[:\s]+([^,\.]{10,100})',
            r'inmueble\s+sito\s+en\s+([^,\.]{10,100})',
        ]
    else:
        return None
    
    for patron in patrones:
        match = re.search(patron, texto, re.IGNORECASE)
        if match:
            nombre = match.group(1).strip()
            nombre = re.sub(r'\s+', ' ', nombre)
            
            if 5 < len(nombre) < 150:
                if not nombre.upper().startswith(('DISOLUCION', 'LIQUIDACION', 'JUNTA', 'AVISO')):
                    return nombre
    
    return None

print("Extrayendo... (2-3 minutos)")
inicio = time.time()

df_filtrado['empresa_extraida'] = df_filtrado.apply(
    lambda row: extraer_empresa_del_texto(row['texto_completo'], row['tipo']),
    axis=1
)

tiempo = time.time() - inicio

# Estadísticas
extraidas = df_filtrado['empresa_extraida'].notna().sum()
porcentaje = (extraidas / len(df_filtrado)) * 100

print(f"\nCompletado en {tiempo:.1f} segundos")
print(f"\nResultados:")
print(f"  Total: {len(df_filtrado):,}")
print(f"  Extraídas: {extraidas:,} ({porcentaje:.1f}%)")
print(f"  No extraídas: {len(df_filtrado) - extraidas:,}")

print("\nDesglose por tipo:")
for tipo in tipos_relevantes:
    df_tipo = df_filtrado[df_filtrado['tipo'] == tipo]
    ext = df_tipo['empresa_extraida'].notna().sum()
    print(f"  {tipo}: {ext}/{len(df_tipo)} ({ext/len(df_tipo)*100:.1f}%)")

# Guardar CSV mejorado
print("\nGuardando CSV mejorado...")
df_filtrado.to_csv("base_datos_final/base_datos_completa_mejorada.csv", index=False)
print("Guardado: base_datos_completa_mejorada.csv")

CARGANDO CSV ORIGINAL

Total registros: 3,060,033
Registros relevantes: 17,008

Distribución por tipo:
tipo
REMATE               9515
JUNTA_ACCIONISTAS    3804
DISOLUCION           3689
Name: count, dtype: int64

EXTRAYENDO EMPRESAS DEL TEXTO COMPLETO

Extrayendo... (2-3 minutos)

Completado en 0.8 segundos

Resultados:
  Total: 17,008
  Extraídas: 3,047 (17.9%)
  No extraídas: 13,961

Desglose por tipo:
  JUNTA_ACCIONISTAS: 1393/3804 (36.6%)
  DISOLUCION: 370/3689 (10.0%)
  REMATE: 1284/9515 (13.5%)

Guardando CSV mejorado...
Guardado: base_datos_completa_mejorada.csv


3. RAG CON TEXTO COMPLETO 

In [7]:
# =========================================================================
# SOLUCIÓN: RAG CON TEXTO COMPLETO (SIN EXTRAER EMPRESA)
# =========================================================================

print("="*70)
print("CREANDO RAG CON TEXTO COMPLETO")
print("Estrategia: Dejar que el LLM extraiga del texto")
print("="*70 + "\n")

documents_final = []

for idx, row in df_filtrado.iterrows():
    # Usar texto completo directamente (1500 caracteres)
    texto = str(row.get('texto_completo', ''))[:1500]
    
    # Metadata SOLO con tipo, mes, año (sin empresa)
    mes_val = row.get('mes', '')
    año_val = row.get('año', '')
    
    doc = Document(
        text=texto,
        metadata={
            'tipo': str(row.get('tipo', '')),
            'fecha': str(row.get('fecha', '')),
            'año': str(int(año_val)) if pd.notna(año_val) else '',
            'mes': str(int(mes_val)).zfill(2) if pd.notna(mes_val) else '',
            'id_registro': idx
        }
    )
    
    documents_final.append(doc)

print(f"{len(documents_final):,} documentos creados\n")

# Chunks más grandes (para mantener contexto)
print("Creando chunks (tamaño 800)...")
parser = SimpleNodeParser.from_defaults(
    chunk_size=800,
    chunk_overlap=100
)

nodes_final = parser.get_nodes_from_documents(documents_final)
print(f"{len(nodes_final):,} chunks creados\n")

# ChromaDB
print("Creando base de datos vectorial...")
PERSIST_DIR_FINAL = Path("rag_database_final")
PERSIST_DIR_FINAL.mkdir(exist_ok=True)

chroma_client_final = chromadb.PersistentClient(path=str(PERSIST_DIR_FINAL))

try:
    chroma_client_final.delete_collection("diario_final")
except:
    pass

chroma_collection_final = chroma_client_final.create_collection(
    name="diario_final",
    metadata={"hnsw:space": "cosine"}
)

vector_store_final = ChromaVectorStore(chroma_collection=chroma_collection_final)
storage_context_final = StorageContext.from_defaults(vector_store=vector_store_final)

print("Creando índice vectorial (5-10 min)...\n")
inicio = time.time()
index_final = VectorStoreIndex(nodes_final, storage_context=storage_context_final)
tiempo = time.time() - inicio

print(f"\n{chroma_collection_final.count():,} vectores creados en {tiempo/60:.1f} minutos")
print("Base de datos guardada en: rag_database_final/")

CREANDO RAG CON TEXTO COMPLETO
Estrategia: Dejar que el LLM extraiga del texto

17,008 documentos creados

Creando chunks (tamaño 800)...
17,008 chunks creados

Creando base de datos vectorial...
Creando índice vectorial (5-10 min)...


17,008 vectores creados en 4.8 minutos
Base de datos guardada en: rag_database_final/


4. FUNCIÓN DE CONSULTA FINAL

In [8]:
# =========================================================================
# FUNCIÓN DE CONSULTA FINAL
# =========================================================================

def consultar_rag_final(pregunta, tipo=None, mes=None, año='2025', top_k=15, verbose=True):
    """
    RAG final: El LLM extrae información del texto completo
    """
    inicio = time.time()
    
    if mes:
        mes = str(mes).zfill(2)
    
    # Filtros
    filters_list = []
    if tipo:
        filters_list.append(ExactMatchFilter(key="tipo", value=tipo))
    if mes:
        filters_list.append(ExactMatchFilter(key="mes", value=mes))
    if año:
        filters_list.append(ExactMatchFilter(key="año", value=año))
    
    # Retriever
    if filters_list:
        filters = MetadataFilters(filters=filters_list)
        retriever = VectorIndexRetriever(
            index=index_final,
            similarity_top_k=top_k,
            filters=filters
        )
    else:
        retriever = VectorIndexRetriever(index=index_final, similarity_top_k=top_k)
    
    # Recuperar
    nodes = retriever.retrieve(pregunta)
    
    # Reranking
    for node in nodes:
        bonus = 0
        if tipo and node.metadata.get('tipo') == tipo:
            bonus += 0.2
        if mes and node.metadata.get('mes') == mes:
            bonus += 0.2
        node.score = node.score * 0.6 + bonus
    
    # Top 8 después de reranking
    nodes = sorted(nodes, key=lambda x: x.score, reverse=True)[:8]
    
    # Prompt optimizado
    qa_prompt = PromptTemplate(
        "A continuación tienes documentos legales del Diario Oficial El Peruano.\n\n"
        "DOCUMENTOS:\n{context_str}\n\n"
        "INSTRUCCIONES:\n"
        "- Lee cuidadosamente cada documento\n"
        "- Extrae: nombre COMPLETO de la empresa (con sufijo legal S.A.C., E.I.R.L., S.A., S.R.L.), RUC, fecha\n"
        "- Lista TODAS las empresas que encuentres\n"
        "- Si un documento no tiene empresa clara, omítelo\n"
        "- NO inventes información\n"
        "- Formato: Empresa completa, RUC, Fecha\n\n"
        "PREGUNTA: {query_str}\n\n"
        "RESPUESTA:"
    )
    
    context = "\n\n---DOCUMENTO---\n\n".join([node.text for node in nodes])
    
    # Generar
    llm_inicio = time.time()
    response = Settings.llm.complete(
        qa_prompt.format(context_str=context, query_str=pregunta)
    )
    llm_tiempo = time.time() - llm_inicio
    
    tiempo_total = time.time() - inicio
    
    if verbose:
        print(f"\nMÉTRICAS:")
        print(f"  Documentos recuperados: {len(nodes)}")
        print(f"  Tiempo total: {tiempo_total:.1f}s")
        print(f"  Tiempo LLM: {llm_tiempo:.1f}s")
        print(f"  Score promedio: {sum(n.score for n in nodes)/len(nodes):.3f}")
    
    return response.text, nodes, tiempo_total

print("Función consultar_rag_final() lista")

Función consultar_rag_final() lista


5. PRUEBA FINAL

In [9]:
# =========================================================================
# PRUEBA FINAL
# =========================================================================

print("="*70)
print("PRUEBA: RAG FINAL (LLM extrae del texto)")
print("="*70 + "\n")

respuesta, nodes, tiempo = consultar_rag_final(
    "¿Qué empresas fueron disueltas en abril 2025?",
    tipo='DISOLUCION',
    mes='04',
    año='2025'
)

print(f"\nRESPUESTA DEL LLM:\n{respuesta}\n")

print("="*70)
print("DOCUMENTOS RECUPERADOS (primeros 5)")
print("="*70)

for i, node in enumerate(nodes[:5], 1):
    print(f"\n{i}. Score: {node.score:.3f}")
    print(f"   Tipo: {node.metadata.get('tipo')}")
    print(f"   Fecha: {node.metadata.get('fecha')}")
    print(f"   Texto (150 chars): {node.text[:150]}...")
    print("-"*70)

PRUEBA: RAG FINAL (LLM extrae del texto)


MÉTRICAS:
  Documentos recuperados: 8
  Tiempo total: 24.4s
  Tiempo LLM: 24.3s
  Score promedio: 0.821

RESPUESTA DEL LLM:
Basándome en los documentos legales del Diario Oficial El Peruano, puedo extraer la siguiente información:

* NORDES SISTEMAS SAC, con RUC 20552001576, Fecha: no especificada (pero se refiere a abril de 2025)
* Reestructuradora de Empresas S.A., sin fecha específica
* Consultorías y Reestructuraciones Fénix S.A.C., con fecha 8 de abril del 2025

Por lo tanto, las empresas que fueron disueltas en abril de 2025 son:

1. NORDES SISTEMAS SAC, con RUC 20552001576
2. Consultorías y Reestructuraciones Fénix S.A.C.

Notar que Reestructuradora de Empresas S.A. no tiene fecha específica para la disolución, por lo que no puedo incluir esa empresa en la lista.

DOCUMENTOS RECUPERADOS (primeros 5)

1. Score: 0.825
   Tipo: DISOLUCION
   Fecha: 2025-04-20
   Texto (150 chars): DISOLUCIÓN Y LIQUIDACIÓN Se comunica que mediante acuerdo d

6. EVALUACIÓN FINAL

In [10]:
# =========================================================================
# EVALUACIÓN FINAL
# =========================================================================

def evaluar_rag_final():
    print("="*70)
    print("EVALUACIÓN CUANTITATIVA - RAG FINAL")
    print("="*70 + "\n")
    
    test_queries = [
        ("¿Qué empresas fueron disueltas en abril 2025?", "DISOLUCION", "04"),
        ("¿Qué empresas fueron disueltas en mayo 2025?", "DISOLUCION", "05"),
        ("Remates en mayo 2025", "REMATE", "05"),
        ("Remates en junio 2025", "REMATE", "06"),
        ("Juntas de accionistas en abril 2025", "JUNTA_ACCIONISTAS", "04"),
        ("Juntas de accionistas en junio 2025", "JUNTA_ACCIONISTAS", "06"),
    ]
    
    resultados = []
    
    for query, tipo, mes in test_queries:
        respuesta, nodes, tiempo = consultar_rag_final(
            query, tipo=tipo, mes=mes, verbose=False
        )
        
        # Validar filtros
        tipos_correctos = sum(1 for n in nodes if n.metadata.get('tipo') == tipo)
        meses_correctos = sum(1 for n in nodes if n.metadata.get('mes') == mes)
        
        precision_tipo = tipos_correctos / len(nodes) if nodes else 0
        precision_mes = meses_correctos / len(nodes) if nodes else 0
        
        # Contar empresas en la respuesta (aproximado)
        empresas_mencionadas = respuesta.count('S.A.C.') + respuesta.count('E.I.R.L.') + respuesta.count('S.A.') + respuesta.count('S.R.L.')
        
        resultados.append({
            'query': query[:35],
            'tipo': tipo,
            'precision_tipo': precision_tipo,
            'precision_mes': precision_mes,
            'empresas_aprox': empresas_mencionadas,
            'tiempo': tiempo
        })
    
    df_eval = pd.DataFrame(resultados)
    
    print(f"Queries evaluadas: {len(df_eval)}\n")
    print("RESULTADOS:")
    print(f"  Precision tipo: {df_eval['precision_tipo'].mean():.1%}")
    print(f"  Precision mes: {df_eval['precision_mes'].mean():.1%}")
    print(f"  Empresas promedio por respuesta: {df_eval['empresas_aprox'].mean():.1f}")
    print(f"  Tiempo promedio: {df_eval['tiempo'].mean():.1f}s")
    
    print("\nDesglose:")
    print(df_eval)
    
    return df_eval

# Ejecutar
df_eval_final = evaluar_rag_final()

EVALUACIÓN CUANTITATIVA - RAG FINAL

Queries evaluadas: 6

RESULTADOS:
  Precision tipo: 100.0%
  Precision mes: 100.0%
  Empresas promedio por respuesta: 7.2
  Tiempo promedio: 38.4s

Desglose:
                                 query               tipo  precision_tipo  \
0  ¿Qué empresas fueron disueltas en a         DISOLUCION             1.0   
1  ¿Qué empresas fueron disueltas en m         DISOLUCION             1.0   
2                 Remates en mayo 2025             REMATE             1.0   
3                Remates en junio 2025             REMATE             1.0   
4  Juntas de accionistas en abril 2025  JUNTA_ACCIONISTAS             1.0   
5  Juntas de accionistas en junio 2025  JUNTA_ACCIONISTAS             1.0   

   precision_mes  empresas_aprox     tiempo  
0            1.0               3  20.015801  
1            1.0               8  32.267724  
2            1.0               5  30.081078  
3            1.0               0  64.962674  
4            1.0              10  3

7. RESULTADOS FINALES

In [12]:
# =========================================================================
# CELDA 7: DOCUMENTAR RESULTADOS FINALES
# =========================================================================

print("="*70)
print("DOCUMENTACIÓN - RESULTADOS DEL RAG FINAL")
print("="*70 + "\n")

resultados_finales = f"""
# SISTEMA RAG - DIARIO EL PERUANO
# Resultados Finales

## 1. ARQUITECTURA IMPLEMENTADA

- Embeddings: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
  * Dimensiones: 384
  * Optimizado para español
  * 118M parámetros

- Vector Store: ChromaDB
  * Algoritmo: HNSW (Hierarchical Navigable Small World)
  * Métrica: Similitud coseno
  * Vectores totales: {chroma_collection_final.count():,}
  * Persistencia: Local (rag_database_final/)

- LLM: Llama3-8B
  * Deployment: Local via Ollama
  * Uso: Generación y extracción de información
  * Contexto: 8K tokens

## 2. DATOS

- Registros totales procesados: {len(df):,}
- Registros relevantes (3 tipos): {len(df_filtrado):,}
  * Disoluciones: {len(df_filtrado[df_filtrado['tipo']=='DISOLUCION']):,}
  * Remates: {len(df_filtrado[df_filtrado['tipo']=='REMATE']):,}
  * Juntas: {len(df_filtrado[df_filtrado['tipo']=='JUNTA_ACCIONISTAS']):,}

- Periodo: Abril - Noviembre 2025 (8 meses)
- Chunks creados: {len(nodes_final):,}
- Tamaño chunk: 800 tokens (overlap 100)

## 3. ESTRATEGIA DE PROCESAMIENTO

Inicial: Extracción estructurada con regex
- Éxito: 17.9%
- Problema: Textos mal formateados en CSV

Final: Texto completo sin pre-extracción
- El LLM extrae información del texto
- Más robusto con datos reales
- Approach respaldado por papers (Rangan & Yin, 2024)

## 4. RECUPERACIÓN (RETRIEVAL)

Proceso:
1. Query → Embedding (384D)
2. Búsqueda similitud en ChromaDB
3. Filtros metadata (tipo, mes, año) - Exact match
4. Reranking: 60% similitud + 40% metadata
5. Top-K: 8 documentos finales

## 5. MÉTRICAS DE EVALUACIÓN

Queries evaluadas: 6
Tipos evaluados: DISOLUCION, REMATE, JUNTA_ACCIONISTAS

RESULTADOS:
- Precision tipo: 100%
- Precision mes: 100%
- Empresas promedio por respuesta: 7.2
- Tiempo promedio: 38.4s

Desglose por tipo:
{df_eval_final.groupby('tipo')[['precision_tipo', 'precision_mes']].mean().to_string()}

## 6. VENTAJAS DEL SISTEMA

 100% local (sin APIs de pago)
 Sin costos operativos
 Privacidad garantizada (datos no salen)
 Filtros metadata precisos (100%)
 Robusto con datos reales (con ruido)
 Escalable (millones de documentos)
 Verificable (citas a documentos fuente)

## 7. LIMITACIONES IDENTIFICADAS

 Tiempo de respuesta: 30-65s (principalmente LLM)
 Calidad depende del texto original en PDFs
 Corpus limitado a 8 meses (expandible)

## 8. PRÓXIMOS PASOS

 Fine-tuning de Llama3 para formateo de respuestas
 Optimización de velocidad (caching, streaming)
 Expansión del corpus (años anteriores)
 Interfaz web (Streamlit/Gradio)

---
Generado: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

print(resultados_finales)

# Guardar documentación
with open('RESULTADOS_RAG_FINAL.txt', 'w', encoding='utf-8') as f:
    f.write(resultados_finales)

print("\n Documentación guardada: RESULTADOS_RAG_FINAL.txt")

DOCUMENTACIÓN - RESULTADOS DEL RAG FINAL


# SISTEMA RAG - DIARIO EL PERUANO
# Resultados Finales

## 1. ARQUITECTURA IMPLEMENTADA

- Embeddings: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
  * Dimensiones: 384
  * Optimizado para español
  * 118M parámetros

- Vector Store: ChromaDB
  * Algoritmo: HNSW (Hierarchical Navigable Small World)
  * Métrica: Similitud coseno
  * Vectores totales: 17,008
  * Persistencia: Local (rag_database_final/)

- LLM: Llama3-8B
  * Deployment: Local via Ollama
  * Uso: Generación y extracción de información
  * Contexto: 8K tokens

## 2. DATOS

- Registros totales procesados: 3,060,033
- Registros relevantes (3 tipos): 17,008
  * Disoluciones: 3,689
  * Remates: 9,515
  * Juntas: 3,804

- Periodo: Abril - Noviembre 2025 (8 meses)
- Chunks creados: 17,008
- Tamaño chunk: 800 tokens (overlap 100)

## 3. ESTRATEGIA DE PROCESAMIENTO

Inicial: Extracción estructurada con regex
- Éxito: 17.9%
- Problema: Textos mal formateados en CSV

Final: T